<a href="https://colab.research.google.com/github/akashgoyal-20/Machine-Learning-Assignments/blob/main/ML_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 selenium pandas lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


Ques 1: Write a Python program to scrape all available books from the website
(https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe,
legal, no anti-bot). For each book, extract the following details:
1. Title
2. Price
3. Availability (In stock / Out of stock)
4. Star Rating (One, Two, Three, Four, Five)
Store the scraped results into a Pandas DataFrame and export them to a CSV file named
books.csv.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

titles=[];prices=[];stocks=[];ratings=[]
url="https://books.toscrape.com/catalogue/page-1.html"
while url:
    r=requests.get(url)
    soup=BeautifulSoup(r.text,"html.parser")
    books=soup.select(".product_pod")
    for b in books:
        titles.append(b.h3.a["title"])
        prices.append(b.select_one(".price_color").text)
        stocks.append(b.select_one(".availability").text.strip())
        ratings.append(b.p["class"][1])
    nxt=soup.select_one(".next a")
    if nxt: url="https://books.toscrape.com/catalogue/"+nxt["href"]
    else: url=None
df=pd.DataFrame({"Title":titles,"Price":prices,"Availability":stocks,"Star Rating":ratings})
df.to_csv("books.csv",index=False)
df.head()

,Title,Price,Availability,Star Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


Ques 2: Write a Python program to scrape the IMDB Top 250 Movies list
(https://www.imdb.com/chart/top/) . For each movie, extract the following details:
1. Rank (1–250)
2. Movie Title
3. Year of Release
4. IMDB Rating
Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv.

In [3]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

browser=webdriver.Chrome(options=chrome_opts)

browser.get("https://www.imdb.com/chart/top/")
time.sleep(5)

film_list=[]
movie_cards=browser.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")

for rank, card in enumerate(movie_cards, start=1):
    try:
        name=card.find_element(By.CSS_SELECTOR,"h3").text
        release_year=card.find_element(By.CSS_SELECTOR,".cli-title-metadata-item").text
        score=card.find_element(By.CSS_SELECTOR,".ipc-rating-star--imdb").text.split()[0]
        film_list.append([rank,name,release_year,score])
    except Exception as e:
        print(f"Skipping a card due to error: {e}")

browser.quit()

imdb_table = pd.DataFrame(film_list, columns=["Position", "Movie", "Release Year", "Rating"])
imdb_table.to_csv("imdb_top250.csv", index=False)
print(imdb_table.head())

   Position                        Movie Release Year Rating
0         1  1. The Shawshank Redemption         1994    9.3
1         2             2. The Godfather         1972    9.2
2         3           3. The Dark Knight         2008    9.1
3         4     4. The Godfather Part II         1974    9.0
4         5              5. 12 Angry Men         1957    9.0


Ques 3: Write a Python program to scrape the weather information for top world cities from the
given website (https://www.timeanddate.com/weather/) . For each city, extract the following
details:
1. City Name
2. Temperature
3. Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)
Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

browser=webdriver.Chrome(options=chrome_opts)

browser.get("https://www.timeanddate.com/weather/")
time.sleep(5)

rows=browser.find_elements(By.CSS_SELECTOR,"table tbody tr")

weather_list=[]
for row in rows[:20]:
    try:
        city=row.find_element(By.CSS_SELECTOR,"a").text
        temp=row.find_elements(By.CSS_SELECTOR,"td")[1].text
        condition=row.find_elements(By.CSS_SELECTOR,"td")[2].text
        weather_list.append([city, temp, condition])
    except Exception as e:
        print(f"Skipping row due to error:{e}")

browser.quit()

# Save as DataFrame
weather_table=pd.DataFrame(weather_list, columns=["City","Temp","Cond."])
weather_table.to_csv("weather.csv", index=False)
print(weather_table.head())


          City          Temp Cond.
0        Accra   Thu 6:09 pm      
1  Addis Ababa   Thu 9:09 pm      
2     Adelaide   Fri 3:39 am      
3      Algiers   Thu 7:09 pm      
4       Almaty  Thu 11:09 pm      
